# Fase de testes 1:

## Arquitetura

Adaptar a arquitetura da aplicação 1 em imagens:

1.   Treinar um simulador com valores geométricos aleatórios de meta átomos elípticos e as suas respectivas transmitância e fase em x e y, calculadas como funções de L_x e L_y;
2.   Salvar o simulador treinado;
2.   Treinar a CPPN e o discriminador(GAN);
5.   Salvar o gerador treinado;
6.   Introduzir o NSGA-II utilizando uma função de custo baseada em minimizar o erro absoluto em relação a um mapa de fase arbitrário;
     - Método Lento: Otimização ponto a ponto;
     - Método Avançado: Otimização da metassuperfície em si;

# Bibliotecas

In [ ]:
!pip install tqdm

In [ ]:
!pip install pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.8/303.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os

from PIL import Image

from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination

import time
from tqdm import tqdm

# Dados de entrada:

Dataframe com os dados geométricos, a transmissão e a fase associadas a cada meta átomo.

In [ ]:
n = 2000

L_x = np.random.uniform(70, 200, n)
L_y = np.random.uniform(70, 200, n)

Transmitancia_x = np.sqrt(L_x**2 + 123)
Transmitancia_y = np.sqrt(L_y**2 + 321)

Fase_x = np.arctan2(L_x + 123, L_y + 321)
Fase_y = np.arctan2(L_x - 123, L_y - 321)

df = pd.DataFrame({
    "L_x": L_x,
    "L_y": L_y,
    "Transmitancia_x": Transmitancia_x,
    "Transmitancia_y": Transmitancia_y,
    "Fase_x": Fase_x,
    "Fase_y": Fase_y,
})

print(df.head())


          L_x         L_y  Transmitancia_x  Transmitancia_y    Fase_x  \
0  199.618646  179.246484       199.926496       180.139673  0.572795   
1  124.416368  149.144975       124.909697       150.217254  0.484431   
2   91.081807  198.155288        91.754540       198.963610  0.391121   
3  113.709224  152.582677       114.248797       153.630965  0.463509   
4  116.502776  160.505102       117.029470       161.501975  0.461569   

     Fase_y  
0  2.646068  
1  3.133351  
2 -2.887388  
3 -3.086483  
4 -3.101132  


Salvar o dataframe para poder verificar o treinamento do Simulador.

In [ ]:
caminho = 'df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt'
df.to_csv(caminho, sep='\t', index=False)

print(f"Arquivo salvo em: {caminho}")

✅ Arquivo salvo em: df_LX_LY_T_F_aleatorios_Teste_Definitivo.txt


Transformar o meta átomo em uma imagem 64x64 para que a rede consiga tratá-lo.

In [ ]:
IMG_SIZE = 64 # 64x64 pixels
MAX_DIM_NM = 520 # O tamanho físico total da célula unitária

def draw_meta_atom_ellipse(L_x_nm, L_y_nm):
    """
    Desenha um meta-átomo de ELIPSE como um array numpy binário.
    """
    scale_factor = IMG_SIZE / MAX_DIM_NM

    a_px = L_x_nm * scale_factor
    b_px = L_y_nm * scale_factor

    if a_px == 0 or b_px == 0:
        return np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)

    x = np.arange(0, IMG_SIZE)
    y = np.arange(0, IMG_SIZE)
    xx, yy = np.meshgrid(x, y)

    center = (IMG_SIZE - 1) / 2.0

    term_x = ((xx - center) / a_px) ** 2
    term_y = ((yy - center) / b_px) ** 2

    mask = (term_x + term_y) <= 1

    img = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)
    img[mask] = 1.0 # Define 1.0 (metal) em todos os pixels dentro da elipse

    return img

print("Gerando imagens de elipse a partir do DataFrame...")
all_images = []
for index, row in df.iterrows():
    img = draw_meta_atom_ellipse(row['L_x'], row['L_y'])
    all_images.append(img)
print(f"{len(all_images)} imagens geradas.")

targets = df[['Transmitancia_x', 'Transmitancia_y', 'Fase_x', 'Fase_y']].values.astype(np.float32)

Gerando imagens de elipse a partir do DataFrame...
2000 imagens geradas.


Fazer um adaptador de dados para o PyTorch. Ele pega suas listas de dados brutos e os "empacota" um por um, convertendo-os no formato exato que o PyTorch entende (Tensores) para que possam ser usados no treinamento do modelo.

In [ ]:
class MetaAtomDataset(Dataset):
    """Dataset que carrega imagens de meta-átomos e seus alvos ópticos."""

    def __init__(self, images_list, targets_array, transform=None):

        self.images = images_list
        self.targets = targets_array
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]

        image = np.expand_dims(image, axis=0)

        # Converte para tensor
        image = torch.from_numpy(image)
        target = torch.from_numpy(target)

        if self.transform:
            image = self.transform(image)

        return image, target

Preparar os dados para o treinamento. Agrupa todos os dados brutos em um único dataset, divide em dois conjuntos: 80% para treino e 20% para validação.

Cria dois "alimentadores" (DataLoader) que entregam esses dados em lotes (batches) para a rede neural, um para treinar (train_loader) e outro para validar (val_loader).

In [ ]:
dataset = MetaAtomDataset(all_images, targets, transform=None)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Construir o simulador (Surrogate):

Definir o bloco de construção fundamental de uma ResNet, Bloco Residual ou ResBlock.

In [ ]:
class ResBlock(nn.Module):
    """
    Define um bloco ResNet básico
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.main_path = nn.Sequential(

            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut_path = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.shortcut_path = nn.Sequential(

                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.main_path(x) + self.shortcut_path(x)
        out = F.relu(out)
        return out

Define a arquitetura completa da sua rede neural Simulator, baseada na ResNet.

In [ ]:
# N_OUTPUTS é quantos valores se quer prever
N_OUTPUTS = 4

class ResNetSimulator(nn.Module):
    """
    Implementação do Simulator baseado em ResNet.
    """
    def __init__(self, in_channels=1, n_outputs=N_OUTPUTS):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = ResBlock(64, 64, stride=1)

        self.layer2 = ResBlock(64, 128, stride=2)

        self.layer3 = ResBlock(128, 256, stride=2)

        self.layer4 = ResBlock(256, 256, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))

        self.head = nn.Sequential(
            nn.Linear(256 * 2 * 2, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, n_outputs)
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.head(out)
        return out

Executar o processo de treinamento e validação. Ao fim, salva o modelo treinado.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ResNetSimulator(in_channels=1, n_outputs=N_OUTPUTS).to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.0002)
epochs = 100

print("Iniciando treinamento do Simulator...")

for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for images, targets in train_loader:
        images, targets = images.to(device), targets.to(device)

        optimizer.zero_grad()

        predictions = model(images)

        loss = criterion(predictions, targets)

        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            predictions = model(images)
            loss = criterion(predictions, targets)
            total_val_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

SIMULATOR_PATH = "simulator_Definitivo.pth"
torch.save(model.state_dict(), SIMULATOR_PATH)
print(f"Modelo Simulator salvo em {SIMULATOR_PATH}")

Iniciando treinamento do Simulator...
Epoch [1/100], Train Loss: 8458.3606, Val Loss: 7249.9519
Epoch [2/100], Train Loss: 4494.6039, Val Loss: 3276.8903
Epoch [3/100], Train Loss: 1219.1706, Val Loss: 637.3405
Epoch [4/100], Train Loss: 392.0873, Val Loss: 458.1169
Epoch [5/100], Train Loss: 344.7070, Val Loss: 339.3218
Epoch [6/100], Train Loss: 311.0608, Val Loss: 498.8787
Epoch [7/100], Train Loss: 220.6891, Val Loss: 330.9034
Epoch [8/100], Train Loss: 111.8109, Val Loss: 130.3179
Epoch [9/100], Train Loss: 55.0312, Val Loss: 53.3747
Epoch [10/100], Train Loss: 37.5499, Val Loss: 29.1728
Epoch [11/100], Train Loss: 27.3190, Val Loss: 68.2227
Epoch [12/100], Train Loss: 20.6910, Val Loss: 5.8581
Epoch [13/100], Train Loss: 17.2715, Val Loss: 10.2476
Epoch [14/100], Train Loss: 18.5822, Val Loss: 20.9120
Epoch [15/100], Train Loss: 11.1766, Val Loss: 28.8636
Epoch [16/100], Train Loss: 13.4716, Val Loss: 14.3631
Epoch [17/100], Train Loss: 12.0865, Val Loss: 6.6603
Epoch [18/100], T

# CPPN e GAN:

Componentes de entrada e ativação para o Gerador CPPN:

In [ ]:
class Sin(nn.Module):
    def forward(self, x): return torch.sin(x)

class Gaussian(nn.Module):
    def forward(self, x): return torch.exp(-x**2)

def make_coordinate_grid_with_r(size, device='cpu'):
    """Cria grade de coordenadas (x, y, r) como no artigo [fonte: 140]."""
    xs = np.linspace(-1, 1, size)
    ys = np.linspace(-1, 1, size)
    xx, yy = np.meshgrid(xs, ys)

    # r = sqrt(x^2 + y^2) [fonte: 140]
    rr = np.sqrt(xx**2 + yy**2)

    # Stack (x, y, r)
    coords = np.stack([xx.ravel(), yy.ravel(), rr.ravel()], axis=-1).astype(np.float32)
    return torch.from_numpy(coords).to(device) # shape (size*size, 3)

Define a arquitetura do gerador (CPPN):

In [ ]:
class CPPN_Generator(nn.Module):
    """
    Gerador CPPN. Mapeia (vetor latente 'v' + coords 'x,y,r') -> pixel.
    """
    def __init__(self, latent_dim, in_coords=3, out_channels=1, hidden_dim=32):
        super().__init__()
        self.latent_dim = latent_dim
        self.in_coords = in_coords

        self.out_channels = out_channels

        input_dim = in_coords + latent_dim

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            Sin(),
            nn.Linear(hidden_dim, hidden_dim),
            Gaussian(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, out_channels),
            nn.Sigmoid()
        )

    def forward(self, coords, v):

        v_expanded = v.unsqueeze(1)
        v_tiled = v_expanded.repeat(1, coords.size(0), 1)

        coords_tiled = coords.unsqueeze(0).repeat(v.size(0), 1, 1)

        combined_input = torch.cat([coords_tiled, v_tiled], dim=-1)

        output = self.net(combined_input)

        img_size = int(np.sqrt(coords.size(0)))

        output = output.permute(0, 2, 1).view(-1, self.out_channels, img_size, img_size)

        return output

Definer o Crítico/Discriminador, a rede neural adversária da GAN. A função dele é olhar para uma imagem de meta-átomo, seja uma real do seu dataset ou uma "falsa" do CPPN_Generator, e dar uma nota que avalia o quão "real" ela parece ser. Ele é uma Rede Neural Convolucional (CNN).

In [ ]:
class CriticCNN(nn.Module):
    """
    Crítico (Discriminador).
    """
    def __init__(self, in_channels=1):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 16, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(16, 32, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
        )

        self.head = nn.Linear(128 * 4 * 4, 1)

    def forward(self, x):
        f = self.features(x)
        f = f.view(x.size(0), -1)
        return self.head(f)

Penalidade de Gradiente (WGAN-GP), treinamento de Redes Adversariais Gerativas (GANs):

In [ ]:
def compute_gradient_penalty(critic, real_samples, fake_samples, device):
    """
    Calcula a penalidade de gradiente (WGAN-GP).
    """
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    critic_interpolates = critic(interpolates)

    gradients = torch.autograd.grad(
        outputs=critic_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(critic_interpolates, device=device),
        create_graph=True,
        retain_graph=True,
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

Função principal de treinamento da Rede Adversarial Gerativa (GAN). O objetivo dela é treinar o CPPN_Generator para que ele aprenda a desenhar imagens de meta-átomos que sejam indistinguíveis das suas imagens reais vindas do DataFrame.

In [ ]:
class MetaAtomImageDataset(Dataset):
    """
    Dataset que carrega apenas as imagens de meta-átomos.
    """

    def __init__(self, images_list):
        self.images = images_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = np.expand_dims(image, axis=0)
        return torch.from_numpy(image)

def train_cppn_gan(all_images_numpy, device):
    """
    Função principal para treinar a CPPN-GAN.
    """
    # Hiperparâmetros baseados no Artigo
    LATENT_DIM = 128
    BATCH_SIZE = 64
    LR = 0.0001
    BETA1 = 0.5
    BETA2 = 0.9
    N_CRITIC = 5
    EPOCHS = 50
    LAMBDA_GP = 10

    dataset = MetaAtomImageDataset(all_images_numpy)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

    coords_grid = make_coordinate_grid_with_r(IMG_SIZE, device=device)

    generator = CPPN_Generator(latent_dim=LATENT_DIM).to(device)
    critic = CriticCNN(in_channels=1).to(device)

    optimizer_G = optim.Adam(generator.parameters(), lr=LR, betas=(BETA1, BETA2))
    optimizer_D = optim.Adam(critic.parameters(), lr=LR, betas=(BETA1, BETA2))

    print("Iniciando treinamento da CPPN-GAN...")

    for epoch in range(EPOCHS):
        for i, real_images in enumerate(dataloader):
            real_images = real_images.to(device)
            current_batch_size = real_images.size(0)

            optimizer_D.zero_grad()

            z = torch.randn(current_batch_size, LATENT_DIM, device=device)
            with torch.no_grad():
                fake_images = generator(coords_grid, z)

            real_score = critic(real_images)
            fake_score = critic(fake_images)

            gp = compute_gradient_penalty(critic, real_images, fake_images, device)

            loss_D = fake_score.mean() - real_score.mean() + LAMBDA_GP * gp

            loss_D.backward()
            optimizer_D.step()

            if (i + 1) % N_CRITIC == 0:
                optimizer_G.zero_grad()

                z = torch.randn(current_batch_size, LATENT_DIM, device=device)
                fake_images_for_G = generator(coords_grid, z)

                fake_score_for_G = critic(fake_images_for_G)

                loss_G = -fake_score_for_G.mean()

                loss_G.backward()
                optimizer_G.step()

        print(f"Epoch [{epoch+1}/{EPOCHS}], "
              f"Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

    GENERATOR_PATH = "generator.pth"
    torch.save(generator.state_dict(), GENERATOR_PATH)
    print(f"Treinamento da GAN concluído. Gerador salvo em {GENERATOR_PATH}")
    return generator

Executar o Script:

In [ ]:
if __name__ == '__main__':

    if 'all_images' not in locals() or len(all_images) == 0:
        print("Erro: A lista 'all_images' não foi encontrada ou está vazia.")

    else:
        print(f"{len(all_images)} imagens reais prontas.")

        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Usando dispositivo: {device}")

        trained_generator = train_cppn_gan(all_images, device=device)

        print("Pré-computação concluída.")

2000 imagens reais prontas.
Usando dispositivo: cpu
Iniciando treinamento da CPPN-GAN...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [1/50], Loss D: 7.7296, Loss G: 0.2196
Epoch [2/50], Loss D: -8.4989, Loss G: 4.8326
Epoch [3/50], Loss D: -31.7827, Loss G: 23.5055
Epoch [4/50], Loss D: -36.4992, Loss G: 29.1156
Epoch [5/50], Loss D: -39.7526, Loss G: 29.3558
Epoch [6/50], Loss D: -42.7740, Loss G: 27.9702
Epoch [7/50], Loss D: -46.4645, Loss G: 20.1706
Epoch [8/50], Loss D: -45.7728, Loss G: 13.7430
Epoch [9/50], Loss D: -45.3402, Loss G: 5.1545
Epoch [10/50], Loss D: -47.8192, Loss G: -1.7008
Epoch [11/50], Loss D: -48.3824, Loss G: -7.6528
Epoch [12/50], Loss D: -48.1306, Loss G: -12.2907
Epoch [13/50], Loss D: -47.6755, Loss G: -15.9324
Epoch [14/50], Loss D: -45.4619, Loss G: -14.8428
Epoch [15/50], Loss D: -45.9126, Loss G: -15.7833
Epoch [16/50], Loss D: -46.0597, Loss G: -18.8815
Epoch [17/50], Loss D: -46.3521, Loss G: -23.5378
Epoch [18/50], Loss D: -46.3112, Loss G: -25.6249
Epoch [19/50], Loss D: -44.7648, Loss G: -26.3507
Epoch [20/50], Loss D: -43.3599, Loss G: -29.6614
Epoch [21/50], Loss D: -43

# Otimização por algoritmo genético

## Método Lento:
Consiste em otimizar átomo por átomo e unir os resultados na metassuperfície final.

A classe abaixo define o problema de otimização para o algoritmo genético (NSGA-II). Configura o problema, informando ao NSGA-II que ele deve evoluir um vetor latente (n_var) para minimizar dois objetivos (n_obj=2).

In [ ]:
class MetaSurface_Problem(Problem):
    """
    Define o problema de otimização da metassuperfície para pymoo (NSGA-II).
    """

    def __init__(self,
                 generator,
                 simulator,
                 target_T_xx,  # <-- Alvo 1: Transmitância X
                 target_T_yy,  # <-- Alvo 2: Transmitância Y
                 target_F_xx,  # <-- Alvo 3: Fase X
                 target_F_yy,  # <-- Alvo 4: Fase Y
                 coords_grid_atom,
                 coords_grid_surface,
                 device):

        self.generator = generator
        self.simulator = simulator
        self.device = device

        # Armazena os 4 alvos como tensores
        self.target_T_xx = torch.tensor(target_T_xx, device=device, dtype=torch.float32)
        self.target_T_yy = torch.tensor(target_T_yy, device=device, dtype=torch.float32)
        self.target_F_xx = torch.tensor(target_F_xx, device=device, dtype=torch.float32)
        self.target_F_yy = torch.tensor(target_F_yy, device=device, dtype=torch.float32)

        self.coords_grid_atom = coords_grid_atom
        self.coords_grid_surface = coords_grid_surface

        LATENT_DIM = generator.latent_dim

        super().__init__(n_var=LATENT_DIM, n_obj=4, n_constr=0, xl=-1.0, xu=1.0)

    def _evaluate(self, X, out, *args, **kwargs):
        pop_size = X.shape[0]
        v_batch = torch.tensor(X, dtype=torch.float32, device=self.device)

        # Arrays para armazenar os 4 erros
        f1_batch = np.zeros(pop_size) # Erro T_x
        f2_batch = np.zeros(pop_size) # Erro T_y
        f3_batch = np.zeros(pop_size) # Erro F_x
        f4_batch = np.zeros(pop_size) # Erro F_y

        with torch.no_grad():
            for i in range(pop_size):
                v = v_batch[i].unsqueeze(0)
                img_tensor_gray = self.generator(self.coords_grid_atom, v)
                img_tensor_binary = (img_tensor_gray > 0.5).float()

                # preds (1, 4) -> [T_x, T_y, F_x, F_y]
                preds = self.simulator(img_tensor_binary)

                # Calcular os 4 erros de componente
                # A ordem DEVE ser a mesma do seu treinamento

                # Erro 1: |T_x_previsto - T_x_alvo|
                f1 = torch.abs(preds[0, 0] - self.target_T_xx)

                # Erro 2: |T_y_previsto - T_y_alvo|
                f2 = torch.abs(preds[0, 1] - self.target_T_yy)

                # Erro 3: |F_x_previsto - F_x_alvo|
                f3 = torch.abs(preds[0, 2] - self.target_F_xx)

                # Erro 4: |F_y_previsto - F_y_alvo|
                f4 = torch.abs(preds[0, 3] - self.target_F_yy)

                f1_batch[i] = f1.item()
                f2_batch[i] = f2.item()
                f3_batch[i] = f3.item()
                f4_batch[i] = f4.item()

        # Retorna a matriz de fitness com 4 colunas
        out["F"] = np.column_stack([f1_batch, f2_batch, f3_batch, f4_batch])

Configurar e executar a otimização NSGA-II, carregar os modelos pré-treinados, definir um objetivo de design específico, e então inicia o algoritmo NSGA2 para "evoluir" soluções por 100 gerações.

In [ ]:
def optimize_pixel(device, generator, simulator, coords_grid_atom,
                   target_T_xx, target_T_yy, target_F_xx, target_F_yy):
    """
    Executa a otimização NSGA-II para um ÚNICO pixel
    e retorna o melhor vetor latente encontrado.
        """

    # Definir o problema para ESTE pixel
    problem = MetaSurface_Problem(
        generator=generator,
        simulator=simulator,
        # Passa os 4 alvos separados
        target_T_xx=target_T_xx,
        target_T_yy=target_T_yy,
        target_F_xx=target_F_xx,
        target_F_yy=target_F_yy,
        coords_grid_atom=coords_grid_atom,
        coords_grid_surface=None,
        device=device
    )

    # Configurar Algoritmo
    algorithm = NSGA2(
        pop_size=30,
        eliminate_duplicates=True
    )

    termination = get_termination("n_gen", 50)

    # Rodar a otimização
    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=1,
                   save_history=False,
                   verbose=False)

    # Encontrar e retornar a MELHOR solução
    # "Melhor" = a solução mais próxima do ponto de erro 4D (0,0,0,0)
    errors = res.F
    norms = np.linalg.norm(errors, axis=1)
    best_index = np.argmin(norms)

    best_latent_vector = res.X[best_index]
    best_errors = res.F[best_index]

    return best_latent_vector, best_errors

Executar o Script de otimização. Um mapa de fase teste é gerado com a mesma fórmula matemática que gerou os dados iniciais.

In [ ]:
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Usando dispositivo: {device}")

    print("Carregando modelos pré-treinados")
    LATENT_DIM = 128
    ATOM_IMG_SIZE = 64

    generator = CPPN_Generator(latent_dim=LATENT_DIM).to(device)
    generator.load_state_dict(torch.load("/content/generator.pth", map_location=device))
    generator.eval()

    simulator = ResNetSimulator(in_channels=1, n_outputs=4).to(device)
    simulator.load_state_dict(torch.load("/content/simulator_Definitivo.pth", map_location=device))
    simulator.eval()

    coords_grid_atom = make_coordinate_grid_with_r(ATOM_IMG_SIZE, device=device)
    print("Modelos carregados.")

    # Gerar mapas de fase alvo com base nas mesmas equações que geraram o dados de entrada
    MAP_H, MAP_W = 10, 10

    print(f"Gerando metassuperfície de {MAP_H}x{MAP_W} pixels...")

    L_x_map = np.linspace(70, 200, MAP_W)
    L_y_map = np.linspace(70, 200, MAP_H)
    LX_grid, LY_grid = np.meshgrid(L_x_map, L_y_map)

    target_map_amp_xx = np.sqrt(LX_grid**2 + 123)
    target_map_amp_yy = np.sqrt(LY_grid**2 + 321)
    target_map_phase_xx = np.arctan2(LX_grid + 123, LY_grid + 321)
    target_map_phase_yy = np.arctan2(LX_grid - 123, LY_grid - 321)

    print("Mapas de fase e amplitude alvo gerados.")

    # Inicializar Estrutura de Saída
    metasurface_design_vectors = np.zeros((MAP_H, MAP_W, LATENT_DIM))
    metasurface_pixel_errors = np.zeros((MAP_H, MAP_W, 4))

    # Loop de Otimização
    start_time = time.time()

    for r in tqdm(range(MAP_H), desc="Otimizando Linhas"):
        for c in range(MAP_W):
            # a. Pegar os 4 alvos para este pixel (r, c)
            target_T_xx = target_map_amp_xx[r, c]
            target_T_yy = target_map_amp_yy[r, c]
            target_F_xx = target_map_phase_xx[r, c]
            target_F_yy = target_map_phase_yy[r, c]

            # b. Chamar a otimização com os 4 alvos
            best_v, best_err = optimize_pixel(
                device,
                generator,
                simulator,
                coords_grid_atom,
                target_T_xx, target_T_yy, target_F_xx, target_F_yy
            )

            # c. Salvar o melhor genoma e os 4 erros
            metasurface_design_vectors[r, c, :] = best_v
            metasurface_pixel_errors[r, c, :] = best_err # best_err (é um array de 4)

    end_time = time.time()
    print(f"\nOtimização de {MAP_H*MAP_W} pixels concluída em {end_time - start_time:.2f} segundos.")

    # Salvar Resultados Finais
    print("Salvando design final...")
    np.save("metasurface_design_latent_vectors_4obj.npy", metasurface_design_vectors)
    np.save("metasurface_pixel_errors_4obj.npy", metasurface_pixel_errors)

    print("Design salvo em 'metasurface_design_latent_vectors_4obj.npy'")

    # Plotar os 4 mapas de erro
    print("Plotando 4 mapas de erro...")
    plt.figure(figsize=(12, 10))

    plt.subplot(2, 2, 1)
    plt.title("Mapa de Erro (Transmitância X)")
    im1 = plt.imshow(metasurface_pixel_errors[:, :, 0], cmap='viridis', aspect='auto')
    plt.colorbar(im1, label="Erro |T_hat - T_target|")

    plt.subplot(2, 2, 2)
    plt.title("Mapa de Erro (Transmitância Y)")
    im2 = plt.imshow(metasurface_pixel_errors[:, :, 1], cmap='viridis', aspect='auto')
    plt.colorbar(im2, label="Erro |T_hat - T_target|")

    plt.subplot(2, 2, 3)
    plt.title("Mapa de Erro (Fase X)")
    im3 = plt.imshow(metasurface_pixel_errors[:, :, 2], cmap='viridis', aspect='auto')
    plt.colorbar(im3, label="Erro |F_hat - F_target| (rad)")

    plt.subplot(2, 2, 4)
    plt.title("Mapa de Erro (Fase Y)")
    im4 = plt.imshow(metasurface_pixel_errors[:, :, 3], cmap='viridis', aspect='auto')
    plt.colorbar(im4, label="Erro |F_hat - F_target| (rad)")

    plt.suptitle("Mapa de Erros da Metassuperfície Final (4 Objetivos)")
    plt.tight_layout()
    plt.show()

Usando dispositivo: cpu
Carregando modelos pré-treinados
Modelos carregados.
Gerando metassuperfície de 10x10 pixels...
Mapas de fase e amplitude alvo gerados.


Otimizando Linhas:   0%|          | 0/10 [01:03<?, ?it/s]


KeyboardInterrupt: 

## Método Avançado

Otimização da superfície em si.

In [ ]:
class CPPN_Generator_Surface(nn.Module):
    """
    Gerador CPPN Avançado.
    Mapeia (vetor latente 'v_superficie' + coords da superfície 'x,y') -> 'v_pixel'

    A saída desta rede é um vetor latente, não um pixel.
    """
    def __init__(self, surface_latent_dim, pixel_latent_dim, in_coords=2, hidden_dim=64):
        super().__init__()
        self.surface_latent_dim = surface_latent_dim
        self.pixel_latent_dim = pixel_latent_dim
        self.in_coords = in_coords

        input_dim = in_coords + surface_latent_dim

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            Sin(),
            nn.Linear(hidden_dim, hidden_dim),
            Gaussian(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, pixel_latent_dim),
            nn.Tanh()
        )

    def forward(self, coords_surface, v_surface):

        v_expanded = v_surface.unsqueeze(1)
        v_tiled = v_expanded.repeat(1, coords_surface.size(0), 1)

        coords_tiled = coords_surface.unsqueeze(0).repeat(v_surface.size(0), 1, 1)

        combined_input = torch.cat([coords_tiled, v_tiled], dim=-1)

        output = self.net(combined_input)

        return output

Função de grade que cria apenas coordenadas (x, y) para o mapa da superfície.

In [ ]:
def make_coordinate_grid_2D(size_h, size_w, device='cpu'):
    """Cria grade de coordenadas (x, y) para a superfície."""
    xs = np.linspace(-1, 1, size_w)
    ys = np.linspace(-1, 1, size_h)
    xx, yy = np.meshgrid(xs, ys)

    coords = np.stack([xx.ravel(), yy.ravel()], axis=-1).astype(np.float32)
    return torch.from_numpy(coords).to(device)

Nova classe Problem. Ela evolui um único vetor v_superficie e avalia o erro médio do mapa 2D inteiro.

In [ ]:
class MetaSurface_Problem_Advanced_4Obj(Problem):
    """
    Define o problema de otimização AVANÇADO (gera superfície inteira)
    para 4 OBJETIVOS (T_x, T_y, F_x, F_y).
    """

    def __init__(self,
                 generator_surface,    # O novo gerador (CPPN -> v_pixel)
                 generator_atom,       # O gerador antigo (CPPN -> imagem)
                 simulator,            # O ResNetSimulator
                 target_map_T_xx,      # Mapa 2D de alvos
                 target_map_T_yy,
                 target_map_F_xx,
                 target_map_F_yy,
                 coords_grid_atom,       # Coords do átomo (64x64, com 'r')
                 coords_grid_surface,    # Coords da superfície (x,y)
                 device):

        self.generator_surface = generator_surface
        self.generator_atom = generator_atom
        self.simulator = simulator
        self.device = device

        # Alvos
        self.target_T_xx_MAP = torch.tensor(target_map_T_xx, device=device, dtype=torch.float32)
        self.target_T_yy_MAP = torch.tensor(target_map_T_yy, device=device, dtype=torch.float32)
        self.target_F_xx_MAP = torch.tensor(target_map_phase_xx, device=device, dtype=torch.float32)
        self.target_F_yy_MAP = torch.tensor(target_map_phase_yy, device=device, dtype=torch.float32)

        self.map_h, self.map_w = self.target_T_xx_MAP.shape
        self.n_pixels = self.map_h * self.map_w

        self.coords_grid_atom = coords_grid_atom
        self.coords_grid_surface = coords_grid_surface

        # Genoma = vetor latente da SUPERFÍCIE
        LATENT_DIM = generator_surface.surface_latent_dim

        # Otimiza 1 vetor (n_var) para 4 objetivos (n_obj)
        super().__init__(n_var=LATENT_DIM, n_obj=4, n_constr=0, xl=-1.0, xu=1.0)

    def _evaluate(self, X, out, *args, **kwargs):
        pop_size = X.shape[0]
        # X é um batch de 'v_superficie'
        v_surface_batch = torch.tensor(X, dtype=torch.float32, device=self.device)

        f1_batch = np.zeros(pop_size) # Erro T_x
        f2_batch = np.zeros(pop_size) # Erro T_y
        f3_batch = np.zeros(pop_size) # Erro F_x
        f4_batch = np.zeros(pop_size) # Erro F_y

        with torch.no_grad():
            for i in range(pop_size):
                v_surface = v_surface_batch[i].unsqueeze(0)

                v_pixel_map = self.generator_surface(self.coords_grid_surface, v_surface)

                v_pixel_batch = v_pixel_map.squeeze(0)

                img_batch = self.generator_atom(self.coords_grid_atom, v_pixel_batch)
                img_batch_binary = (img_batch > 0.5).float()

                preds_batch = self.simulator(img_batch_binary)

                # Calcular Erro do mapa
                preds_map_T_x = preds_batch[:, 0].view(self.map_h, self.map_w)
                preds_map_T_y = preds_batch[:, 1].view(self.map_h, self.map_w)
                preds_map_F_x = preds_batch[:, 2].view(self.map_h, self.map_w)
                preds_map_F_y = preds_batch[:, 3].view(self.map_h, self.map_w)

                # Calcular o ERRO MÉDIO sobre o mapa inteiro
                f1 = torch.abs(preds_map_T_x - self.target_T_xx_MAP).mean()
                f2 = torch.abs(preds_map_T_y - self.target_T_yy_MAP).mean()
                f3 = torch.abs(preds_map_F_x - self.target_F_xx_MAP).mean()
                f4 = torch.abs(preds_map_F_y - self.target_F_yy_MAP).mean()

                f1_batch[i] = f1.item()
                f2_batch[i] = f2.item()
                f3_batch[i] = f3.item()
                f4_batch[i] = f4.item()

        out["F"] = np.column_stack([f1_batch, f2_batch, f3_batch, f4_batch])

Executar. O mapa de fase foi gerado com base nas equações usada para produzir os dados de entrada.

In [ ]:
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Usando dispositivo: {device}")

    # Parâmetros da Arquitetura
    PIXEL_LATENT_DIM = 128
    SURFACE_LATENT_DIM = 128
    ATOM_IMG_SIZE = 64
    MAP_H, MAP_W = 10, 10
    N_PIXELS = MAP_H * MAP_W

    # Carregar Modelos Pré-Treinados
    print("Carregando modelos pré-treinados...")

    # Carregar Gerador de ÁTOMO
    generator_atom = CPPN_Generator(latent_dim=PIXEL_LATENT_DIM).to(device)
    generator_atom.load_state_dict(torch.load("/content/generator.pth", map_location=device))
    generator_atom.eval()

    # Carregar Simulador
    simulator = ResNetSimulator(in_channels=1, n_outputs=4).to(device)
    simulator.load_state_dict(torch.load("/content/simulator_Definitivo.pth", map_location=device))
    simulator.eval()

    # Novo Gerador de Superfície
    generator_surface = CPPN_Generator_Surface(
        surface_latent_dim=SURFACE_LATENT_DIM,
        pixel_latent_dim=PIXEL_LATENT_DIM
    ).to(device)
    generator_surface.eval()

    print("Modelos carregados e inicializados.")

    # Gerar Coordenadas
    coords_grid_atom = make_coordinate_grid_with_r(ATOM_IMG_SIZE, device=device)
    coords_grid_surface = make_coordinate_grid_2D(MAP_H, MAP_W, device=device)

    # Gerar Mapas de Fase Alvo
    print(f"Gerando mapas alvo {MAP_H}x{MAP_W}...")

    L_x_map = np.linspace(70, 200, MAP_W)
    L_y_map = np.linspace(70, 200, MAP_H)
    LX_grid, LY_grid = np.meshgrid(L_x_map, L_y_map)

    target_map_amp_xx = np.sqrt(LX_grid**2 + 123)
    target_map_amp_yy = np.sqrt(LY_grid**2 + 321)
    target_map_phase_xx = np.arctan2(LX_grid + 123, LY_grid + 321)
    target_map_phase_yy = np.arctan2(LX_grid - 123, LY_grid - 321)

    # Rodar a Otimização
    problem = MetaSurface_Problem_Advanced_4Obj(
        generator_surface=generator_surface,
        generator_atom=generator_atom,
        simulator=simulator,
        target_map_T_xx = target_map_amp_xx,
        target_map_T_yy = target_map_amp_yy,
        target_map_F_xx = target_map_phase_xx,
        target_map_F_yy = target_map_phase_yy,
        coords_grid_atom=coords_grid_atom,
        coords_grid_surface=coords_grid_surface,
        device=device
    )

    algorithm = NSGA2(
        pop_size=20,
        eliminate_duplicates=True
    )

    termination = get_termination("n_gen", 5)

    print("Iniciando otimização AVANÇADA (NSGA-II)...")
    start_time = time.time()
    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=1,
                   save_history=True,
                   verbose=True)
    end_time = time.time()
    print(f"\nOtimização concluída em {end_time - start_time:.2f} segundos.")

    # Salvar e Plotar Resultados
    print("\n--- Resultados da Otimização ---")

    # Encontrar o melhor indivíduo (mais próximo de 0,0,0,0)
    errors = res.F
    norms = np.linalg.norm(errors, axis=1)
    best_index = np.argmin(norms)
    best_v_surface = res.X[best_index]
    best_errors = res.F[best_index]

    print(f"Melhor indivíduo (v_superficie) encontrado com Erros Médios:")
    print(f"  Erro T_x: {best_errors[0]:.4f}")
    print(f"  Erro T_y: {best_errors[1]:.4f}")
    print(f"  Erro F_x: {best_errors[2]:.4f}")
    print(f"  Erro F_y: {best_errors[3]:.4f}")

    # Visualizar o Melhor Design de Metassuperfície
    print("Gerando visualização do melhor design...")
    os.makedirs("otimizacao_avancada_4obj", exist_ok=True)

    # Salvar o vetor "genoma" da superfície vencedora
    np.save("otimizacao_avancada_4obj/best_surface_vector.npy", best_v_surface)

    # Gerar o design final
    with torch.no_grad():
        v_tensor = torch.tensor(best_v_surface, dtype=torch.float32, device=device).unsqueeze(0)

        # 1. Gerar mapa de vetores
        v_pixel_map = generator_surface(coords_grid_surface, v_tensor)
        v_pixel_batch = v_pixel_map.squeeze(0)

        # 2. Gerar batch de imagens
        img_batch = generator_atom(coords_grid_atom, v_pixel_batch) # (N_PIXELS, 1, 64, 64)
        img_batch_binary = (img_batch > 0.5)

        # Salvar o mapa 2D de vetores latentes (o design real)
        design_vectors_2d = v_pixel_batch.cpu().numpy().reshape((MAP_H, MAP_W, PIXEL_LATENT_DIM))
        np.save("otimizacao_avancada_4obj/metasurface_design_vectors_MAP.npy", design_vectors_2d)

        # Criar uma imagem "mosaico" da metassuperfície
        print("Criando imagem-mosaico da metassuperfície...")
        images_np = img_batch_binary.squeeze(1).cpu().numpy()

        canvas = np.zeros((MAP_H * ATOM_IMG_SIZE, MAP_W * ATOM_IMG_SIZE))

        idx = 0
        for r in range(MAP_H):
            for c in range(MAP_W):
                img = images_np[idx]
                r_start, c_start = r * ATOM_IMG_SIZE, c * ATOM_IMG_SIZE
                canvas[r_start:r_start+ATOM_IMG_SIZE, c_start:c_start+ATOM_IMG_SIZE] = img
                idx += 1

        pil_img = Image.fromarray((canvas * 255).astype(np.uint8), mode='L')
        pil_img.save("otimizacao_avancada_4obj/metasurface_mosaico.png")
        print("Design salvo em 'otimizacao_avancada_4obj/metasurface_mosaico.png'")

Usando dispositivo: cpu
Carregando modelos pré-treinados...
Modelos carregados e inicializados.
Gerando mapas alvo 10x10...
Iniciando otimização AVANÇADA (NSGA-II)...
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       20 |     20 |             - |             -
     2 |       40 |     20 |  0.000000E+00 |             f
     3 |       60 |     20 |  0.000000E+00 |             f
     4 |       80 |     20 |  0.000000E+00 |             f
     5 |      100 |     20 |  0.000000E+00 |             f

Otimização concluída em 554.88 segundos.

--- Resultados da Otimização ---
Melhor indivíduo (v_superficie) encontrado com Erros Médios:
  Erro T_x: 88.0284
  Erro T_y: 92.5279
  Erro F_x: 0.4013
  Erro F_y: 5.4173
Gerando visualização do melhor design...
Criando imagem-mosaico da metassuperfície...
Design salvo em 'otimizacao_avancada_4obj/metasurface_mosaico.png'


/tmp/ipython-input-1510451563.py:148: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil_img = Image.fromarray((canvas * 255).astype(np.uint8), mode='L')
